In [41]:
import pandas as pd
import numpy as np
import zipfile
import os
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import re
import nltk
from wordcloud import WordCloud
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datetime import datetime, date
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import coo_matrix
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy import sparse
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from scipy.stats import randint as sp_randint
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import pickle
import joblib
from IPython.display import Image
# tokenisation

In [44]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [6]:
# https://www.kaggle.com/davidgasquez/ndcg-scorer

def dcg_score(y_true, y_score, k=5):
    
    
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


def ndcg_score(ground_truth, predictions, k=5):
    
    
    lb = LabelBinarizer()
    lb.fit(range(predictions.shape[1] + 1))
    T = lb.transform(ground_truth)

    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores)


# NDCG Scorer function
ndcg_scorer = make_scorer(ndcg_score, needs_proba=True, k=5)


In [37]:
def age_mode(age):        #function to replace outliers with mode
    
    
  if age < 18.0 or age > 100.0: 
        
        return 29.0
  
  else: 
        
        return age


In [45]:
# loading from memory
y = np.load('/content/drive/MyDrive/y.npy',allow_pickle=True)

with open("/content/drive/MyDrive/col_lst.txt", "rb") as fp:
    
   col_lst = pickle.load(fp)
    
fp.close()

with open("/content/drive/MyDrive/lst_ohe_train.txt", "rb") as fp:
    
    lst_ohe_train = pickle.load(fp)
    
fp.close()

vectorizer_action = pickle.load(open('/content/drive/MyDrive/vectorizer_action.pickle', 'rb'))

vectorizer_action_type = pickle.load(open("/content/drive/MyDrive/vectorizer_action_type.pickle", "rb"))

vectorizer_action_detail = pickle.load(open("/content/drive/MyDrive/vectorizer_action_detail.pickle", "rb"))


session_df_concat = pd.read_pickle('/content/drive/MyDrive/session_df_concat.pickle')

train = pd.read_pickle('/content/drive/MyDrive/train_raw.pickle')


clf = joblib.load('/content/drive/MyDrive/xgb')  # Our pretrained Xgboost Model

In [11]:
le = LabelEncoder()
# Encoding country 
y = le.fit_transform(y)

y

array([11,  7,  7, ...,  7,  7,  7])

In [62]:
lst_ohe = ['gender', 'signup_method', 'language', 'affiliate_channel',\
            'affiliate_provider', 'signup_app',  'first_browser']

raw_df = train[:2]             #Checking for first 2 rows from training data

raw_label = y[:2]              #Taking corresponding y values
raw_df.columns

Index(['user_id', 'gender', 'age', 'signup_method', 'signup_flow', 'language',
       'affiliate_channel', 'affiliate_provider', 'signup_app',
       'first_device_type', 'first_browser', 'country_destination', 'action',
       'action_type', 'action_detail', 'secs_elapsed', 'account_created_day',
       'account_created_month', 'first_booking_day', 'first_booking_month'],
      dtype='object')

In [46]:
train.columns

Index(['user_id', 'gender', 'age', 'signup_method', 'signup_flow', 'language',
       'affiliate_channel', 'affiliate_provider', 'signup_app',
       'first_device_type', 'first_browser', 'country_destination', 'action',
       'action_type', 'action_detail', 'secs_elapsed', 'account_created_day',
       'account_created_month', 'first_booking_day', 'first_booking_month'],
      dtype='object')

In [65]:
def f1(train):    # This function takes X i.e raw data and performs encoding of data and gets prediction of data from pretrained model

 X= train.drop(['country_destination'], axis=1)
 data_action_tfidf = vectorizer_action.transform(X['action'].values)
 data_action_type_tfidf = vectorizer_action_type.transform(X['action_type'].values)
 data_action_detail_tfidf = vectorizer_action_detail.transform(X['action_detail'].values)
 X= train.drop(['country_destination'], axis=1)
 countries = []
 X['user_id'].fillna('na' , inplace=True)
 X['action'].fillna('na' , inplace=True)
 X['action_type'].fillna('na' , inplace=True)
 X['action_detail'].fillna('na' , inplace=True)
 X['first_device_type'].fillna('na' , inplace=True)
 X['secs_elapsed'].fillna(0, inplace=True)
 X['age'] = X['age'].apply(age_mode)
 X['age'].fillna(29.0 , inplace=True)
 X.drop(['user_id'],axis=1,inplace = True)
    
 for i in range(len(lst_ohe)):
        
        ohe = pd.get_dummies(X[lst_ohe[i]], prefix=lst_ohe[i])
        
        X.drop([lst_ohe[i]], axis=1, inplace = True)
        
        # Get missing columns in the training test
    
        missing_cols = set( lst_ohe_train[i] ) - set( ohe.columns )
    
        # Add a missing column in test set with default value equal to 0
    
        for c in missing_cols:
            
            ohe[c] = 0
        
        #To make sur that order of column in the test set is in the same order than in train set
    
        ohe = ohe[lst_ohe_train[i]]
    
        data = pd.concat((X, ohe), axis=1)
        
 #ohe = X['first_device_type'].str.get_dummies(sep=",")
    
 #X.drop(['first_device_type'], axis=1, inplace = True)
    
    # Get missing columns in the training test
    
 missing_cols = set( lst_ohe_train[-1] ) - set( ohe.columns )
    
    # Add a missing column in test set with default value equal to 0
    
 for c in missing_cols:
        
      ohe[c] = 0
        
    # Ensure the order of column in the test set is in the same order than in train set
    
      ohe = ohe[lst_ohe_train[-1]]
    
      X = pd.concat((data, ohe), axis=1)    
    
 data_action_tfidf = vectorizer_action.transform(X['action'].values)
    
 data_action_type_tfidf = vectorizer_action_type.transform(X['action_type'].values)
        
 data_action_detail_tfidf = vectorizer_action_detail.transform(X['action_detail'].values)
        
 X.drop(['action','action_type','action_detail'],axis=1,inplace = True)
    
 data_tfidf = hstack((X,data_action_tfidf,data_action_type_tfidf,data_action_detail_tfidf)).tocsr()



 from xgboost import XGBClassifier
 pred = xgb.predict_proba(data_tfidf)   
 for i in pred:
        
        countries.append(le.inverse_transform(np.argsort(i)[::-1][:5]).tolist())
    
 return countries,pred


In [68]:
# func2 to output score depending on prediction and actual value

def func2(label,pred):
    
    """
    
    Function takes label and pred as input and returns ndcg score.
    
    parameters:  label , pred
        
    """
    
    score = ndcg_score(label,pred,5)
    
    return score


In [67]:
country,pred = f1(train[:2])

[['US', 'NDF', 'other', 'GB', 'IT'], ['NDF', 'US', 'other', 'FR', 'GB']] [[ 0.07227261  0.07268214  0.07244734  0.0727839   0.0736526   0.07426566
   0.07396927  0.11341434  0.07238723  0.07212908  0.12122008  0.10877571]
 [ 0.06988983  0.07025965  0.07004832  0.0703583   0.0706948   0.07053546
   0.07049852  0.2142804   0.07002889  0.06979483  0.08207586  0.07153518]]


In [69]:
score = func2(y,pred)

score

0.75
